In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("../champ_result/all_beta_normalized.csv")

In [3]:
id_list = pd.read_csv("../result/id_list.txt", header=None)

In [4]:
train = train[train["Unnamed: 0"].isin(id_list[0])]

In [5]:
train_normal = train.iloc[:, 1:95:2].T
train_tumor = train.iloc[:,95::2].T

##### Calculate Δβ

In [6]:
# remove outlier based on every column
def IQR(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    upper_fence = Q3 + IQR*1.5
    lower_fence = Q1 - IQR*1.5
    return upper_fence,lower_fence
def no_outlier(df):
    upper_fence, lower_fence = IQR(df)
    ddf=df[(df>lower_fence)&(df<upper_fence)]
    return ddf

In [7]:
train_normal = no_outlier(train_normal)
train_tumor = no_outlier(train_tumor)

In [8]:
train_normal_avg = train_normal.mean(skipna=True, axis = 0)

In [9]:
train_tumor=(train_tumor).subtract(train_normal_avg, axis = 1)

In [10]:
train_tumor=no_outlier(train_tumor)

In [11]:
train_tumor_mean = train_tumor.mean(skipna=True, axis = 0)

In [12]:
Δβ = pd.merge(train.iloc[:,:1], pd.DataFrame(train_tumor_mean, columns = ["dbeta"]), left_index=True, right_index=True)
Δβ

,Unnamed: 0,dbeta
0,cg00000957,0.017616
1,cg00001349,0.032420
2,cg00001583,0.316867
3,cg00002028,0.014177
4,cg00002837,0.120412
...,...,...
349415,cg27656573,0.003831
349416,cg27657363,-0.013388
349417,cg27657537,0.038450
349418,cg27662611,0.003935


In [13]:
dmp_train = pd.read_csv("../champ_result/DMP_result.csv")
print(f"raw train shape: {dmp_train.shape}")
dmp_train = dmp_train[["Unnamed: 0", "gene"]]
dmp_train.dropna(inplace=True)
print(f"after dropna: {dmp_train.shape}")

raw train shape: (268112, 24)
after dropna: (206165, 2)


In [14]:
result = pd.merge(Δβ, dmp_train, on="Unnamed: 0", how="inner")

In [15]:
def find_max_dBeta_grouped(group):
    idx_max = group['dbeta'].abs().idxmax()
    return group.loc[idx_max]

result_max_per_gene = result.groupby("gene").apply(find_max_dBeta_grouped).reset_index(drop=True)

In [16]:
single = pd.read_csv('../comorbidity/matchgene174_single_3Y10__OR2.txt', sep='\t', header=None)

In [17]:
result_max_per_gene_single = result_max_per_gene[result_max_per_gene['gene'].isin(single[0])]
result_max_per_gene_single

,Unnamed: 0,dbeta,gene
3,cg00134295,0.207143,A2M
4,cg15769388,-0.143162,A2ML1
11,cg13001012,-0.056940,AADAC
14,cg00150882,0.157614,AADAT
15,cg20940607,0.009114,AAGAB
...,...,...,...
18285,cg07135797,-0.288723,ZNRD1
18288,cg20080983,-0.261198,ZNRF3
18304,cg14231297,0.401591,ZSCAN18
18324,cg14642833,-0.018417,ZWINT


In [18]:
result_max_per_gene_single.to_csv("../result/result_max_per_gene_single.csv", index=False)

In [23]:
check = result_max_per_gene_single.loc[result_max_per_gene_single["dbeta"].abs() > 0.35]
check

,Unnamed: 0,dbeta,gene
91,cg13670057,0.359170,ABL1
146,cg10266490,0.403402,ACOT11
151,cg10976772,0.365513,ACOT7
196,cg18770350,0.375086,ACTN2
210,cg09499849,0.383661,ACVR1
...,...,...,...
17772,cg04738965,0.408686,ZIC1
17774,cg16768018,0.462100,ZIC4
17775,cg20985450,0.402938,ZIC5
18045,cg21183523,0.381336,ZNF521
